In [24]:
import sys
import pandas as pd
import Bio
import re
from Bio import Entrez
import urllib.request
import os
from os.path import exists
from ftplib import FTP
import gzip
import shutil
import argparse
from pathlib import Path

ncbi_branches = ["bacteria", "archaea", "viral", "plasmid", "fungi", "protozoa", "invertebrate", "plant", "vertebrate_mammalian", "vertebrate_other"]

# names = []

ncbi_ids = [77608, 198252, 267375]

output = 'C:/Users/willn/Documents/GitHub/biobakery_workflows/biobakery_workflows/assembly_workflow/example/'

In [25]:
for tax_id in ncbi_ids:
    handle = Entrez.efetch(db = "taxonomy", id = str(tax_id), retmode="xml", rettype="gb")
    lines = handle.readlines()
    lines = [line.rstrip() for line in lines]
    str_match = [str(s) for s in lines if "ScientificName" in str(s)]
    if len(str_match) > 0:
        print(re.search('<ScientificName>(.*)</ScientificName>', str_match[0]).group(1))
        name = re.search('<ScientificName>(.*)</ScientificName>', str_match[0]).group(1)
    else:
        raise ValueError('Taxonomic name not present in NCBI lookup')

    print(name)
        
    ftp = FTP('ftp.ncbi.nih.gov')
    ftp.login()

    branch_index = 0
    while True:
        if branch_index == len(ncbi_branches):
            raise ValueError("Genome not found")
        branch = ncbi_branches[branch_index]
        try:
            ftp.cwd('genomes/refseq/' + branch + '/' + name.replace(" ", "_") + '/')
            dir_list = ftp.nlst()
            ftp.close()
            if 'reference' in dir_list:
                ftp = FTP('ftp.ncbi.nih.gov')
                ftp.login()
                ftp.cwd('genomes/refseq/' + branch + '/' + name.replace(" ", "_") + '/reference/')
                strains = ftp.nlst()
                ftp.close()
                genome_type = 'reference'
            elif 'representative' in dir_list:
                ftp = FTP('ftp.ncbi.nih.gov')
                ftp.login()
                ftp.cwd('genomes/refseq/' + branch + '/' + name.replace(" ", "_") + '/representative/')
                strains = ftp.nlst()
                ftp.close()
                genome_type = 'representative'
            break
        except:
            branch_index+=1

        print("No genome found")
        strains = []

    for strain in strains:
        if not exists(output + "genomes/" + strain + "_genomic.fna"):
            print("Retrieving from ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/" + branch + "/" + name.replace(" ", "_") + "/" + genome_type + "/" + strain + "/" + strain + "_genomic.fna")
            try:
                urllib.request.urlretrieve("ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/" + branch + "/" + name.replace(" ", "_") + "/" + genome_type + "/" + strain + "/" + strain + "_genomic.fna.gz", output + "genomes/" + strain + "_genomic.fna.gz")
            except:
                print("Failed, trying again...")
                urllib.request.urlretrieve("ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/" + branch + "/" + name.replace(" ", "_") + "/" + genome_type + "/" + strain + "/" + strain + "_genomic.fna.gz", output + "genomes/" + strain + "_genomic.fna.gz")

Pseudoalteromonas distincta
Pseudoalteromonas distincta
Retrieving from ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Pseudoalteromonas_distincta/representative/GCF_005877035.1_ASM587703v1/GCF_005877035.1_ASM587703v1_genomic.fna
Candidatus Pelagibacter ubique
Candidatus Pelagibacter ubique
Retrieving from ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Candidatus_Pelagibacter_ubique/representative/GCF_000012345.1_ASM1234v1/GCF_000012345.1_ASM1234v1_genomic.fna
Pseudoalteromonas marina
Pseudoalteromonas marina
Retrieving from ftp://ftp.ncbi.nlm.nih.gov/genomes/refseq/bacteria/Pseudoalteromonas_marina/representative/GCF_900141965.1_IMG-taxon_2703719062_annotated_assembly/GCF_900141965.1_IMG-taxon_2703719062_annotated_assembly_genomic.fna
